In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# to handle datasets
import pandas as pd
import numpy as np

# for plotting
import matplotlib.pyplot as plt

# to build the models
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

# to visualise al the columns in the dataframe
pd.pandas.set_option('display.max_columns', None)

In [0]:
# load the train and test set with the engineered variables

X_train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/DVF/data/xtrain.csv')
X_test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/DVF/data/xtest.csv')

X_train.head()

,Date mutation,Nature mutation,Valeur fonciere,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune,Section,Type local,Surface reelle bati,Nombre pieces principales,Nature culture,Surface terrain,No voie_na,Surface reelle bati_na,Nombre pieces principales_na,Surface terrain_na
0,0.854396,0.666667,8.987197,0.002501,0.500,1.0,0.0,0.0,0.159091,0.0,0.347826,0.666667,0.394355,0.045455,0.818182,0.342042,0.0,0.0,0.0,0.0
1,0.574176,0.666667,11.652687,0.000900,0.500,1.0,0.0,0.0,0.068182,0.0,0.434783,1.000000,0.606931,0.030303,0.909091,0.617986,0.0,0.0,0.0,1.0
2,0.502747,0.666667,10.404263,0.008802,0.875,1.0,0.0,0.0,0.113636,0.0,0.434783,1.000000,0.194712,0.000000,0.909091,0.617986,0.0,0.0,0.0,1.0
3,0.428571,0.666667,12.409013,0.003001,0.875,1.0,0.0,0.0,0.954545,0.0,0.782609,0.666667,0.504666,0.045455,0.818182,0.295356,0.0,0.0,0.0,0.0
4,0.420330,0.666667,12.386309,0.004101,0.875,1.0,0.0,0.0,0.568182,0.0,0.434783,0.000000,0.552229,0.024448,0.818182,0.396259,0.0,1.0,1.0,0.0


In [0]:
# capture the target (remember that the target is log transformed)
y_train = X_train['Valeur fonciere']
y_test = X_test['Valeur fonciere']

# drop unnecessary variables from our training and testing sets
X_train.drop(['Valeur fonciere'], axis=1, inplace=True)
X_test.drop(['Valeur fonciere'], axis=1, inplace=True)

### Feature Selection

Let's go ahead and select a subset of the most predictive features. There is an element of randomness in the Lasso regression, so remember to set the seed.

In [0]:
# We will do the model fitting and feature selection
# altogether in a few lines of code

# first, we specify the Lasso Regression model, and we
# select a suitable alpha (equivalent of penalty).
# The bigger the alpha the less features that will be selected.

# Then we use the selectFromModel object from sklearn, which
# will select automatically the features which coefficients are non-zero
from sklearn.ensemble import RandomForestRegressor
# remember to set the seed, the random state in this function
sel_ = SelectFromModel(RandomForestRegressor(n_estimators=70, max_features='sqrt', random_state=0))

# train Lasso model and select features
sel_.fit(X_train, y_train)

SelectFromModel(estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                                criterion='mse', max_depth=None,
                                                max_features='sqrt',
                                                max_leaf_nodes=None,
                                                max_samples=None,
                                                min_impurity_decrease=0.0,
                                                min_impurity_split=None,
                                                min_samples_leaf=1,
                                                min_samples_split=2,
                                                min_weight_fraction_leaf=0.0,
                                                n_estimators=70, n_jobs=None,
                                                oob_score=False, random_state=0,
                                                verbose=0, warm_start=False),
                max_features=None, n

In [0]:
# let's visualise those features that were selected.

sel_.get_support()

array([ True, False, False, False, False, False, False,  True, False,
        True, False, False, False,  True,  True, False, False, False,
       False])

In [0]:
# let's print the number of total and selected features

# this is how we can make a list of the selected features
selected_feats = X_train.columns[(sel_.get_support())]

# let's print some stats
print('total features: {}'.format((X_train.shape[1])))
print('selected features: {}'.format(len(selected_feats)))
print('features with coefficients shrank to zero: {}'.format(
    np.sum(sel_.estimator_.coef_ == 0)))

total features: 19
selected features: 5


AttributeError: ignored

In [0]:
# print the selected features
selected_feats

Index(['Date mutation', 'Code departement', 'Section', 'Nature culture',
       'Surface terrain'],
      dtype='object')

In [0]:
pd.Series(selected_feats).to_csv('/content/drive/My Drive/Colab Notebooks/DVF/data/selected_features.csv', index=False)